In [1]:
!pip install openai
!pip install langchain
!pip install huggingface_hub


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [12]:
import os
os.environ['OPENAI_API_KEY'] = ""
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [14]:
from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(model="gpt-4")

In [15]:
prompt = "Tell me a short story about BMW"
print(llm.invoke(prompt))

content='Title: "The Ultimate Driving Machine"\n\nOnce upon a time, in the heart of Germany, there was a small, struggling aircraft company named Bayerische Flugzeugwerke, founded in 1916. This small company had a bigger dream, a dream not limited to the sky, but to conquer the roads as well.\n\nIn 1917, the company was renamed Bayerische Motoren Werke (BMW), reflecting its new focus on motor vehicle manufacturing. However, the Treaty of Versailles in 1919 banned the production of aircraft engines in Germany, forcing BMW to shift its focus onto the production of motorcycles and eventually, cars.\n\nIn 1928, BMW ventured into car production when they bought Automobilwerk Eisenach, which manufactured Dixi cars. The first car launched after this acquisition was a modified version of Dixi 3/15, named BMW 3/15. The car was well-received, marking BMW\'s successful foray into the automobile industry.\n\nOver the years, BMW rose through the ranks, with its unique blend of luxury, performance, 

In [105]:
from langchain import HuggingFaceHub

llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature": 0.7})
print(f"flan-t5-xxl: {llm('Who won the FIFA World Cup in the year 1994? ')}")

llm = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature": 0.7})
print(f"flan-t5-large: {llm('Who won the FIFA World Cup in the year 1994? ')}")

flan-t5-xxl: brazil
flan-t5-large: argentina


**Prompt templates**

In [106]:
from langchain.prompts import PromptTemplate

prompt_template_lecture_name = PromptTemplate(
    input_variables=["subject"],
    template="I want to write a new lecture about {subject}, suggest a fancy name for this"
)

text = prompt_template_lecture_name.format(subject="Cars")
print(text)
print(llm(text))

I want to write a new lecture about Cars, suggest a fancy name for this
The History of Cars


In [107]:
from langchain import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""

repo_id="google/flan-t5-large"
llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 64}
)
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
print(f"flan-t5-large: {llm_chain.run('Who won the FIFA World Cup in the year 1994? ')}")

llm=ChatOpenAI(model="gpt-4", openai_api_base="http://localhost:1234/v1", max_tokens=5000)
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
print(f"local: {llm_chain.run('Who won the FIFA World Cup in the year 1994? ')}")



flan-t5-large: The 1994 FIFA World Cup was won by Brazil. Brazil won the 1994 FIFA World Cup. So the answer is Brazil.
local:  Step 1: Focus on the year 1994.
Step 2: Narrow it down to a specific group of people who could have potentially won the FIFA World Cup that year.
Step 3: Identify the winner from that list.

The team that won the FIFA World Cup in the year 1994 was Brazil. They defeated Argentina 2-0 in the final match held in Pasadena, United States.


In [25]:
from langchain.chains import LLMChain

llm = ChatOpenAI(model="gpt-4")


# 1
prompt_template_lecture_name = PromptTemplate(
    input_variables=["subject"],
    template="I want to write a new lecture about {subject}, suggest a fancy name for this",
)

chain1 = LLMChain(llm=llm, prompt=prompt_template_lecture_name, verbose=True)


# 2
prompt_template_topics = PromptTemplate(
    input_variables=["title"], template="Suggest some topics for this lecture: {title}"
)

chain2 = LLMChain(llm=llm, prompt=prompt_template_topics, verbose=True)

# seq chain

from langchain.chains import SimpleSequentialChain

schain = SimpleSequentialChain(chains=[chain1, chain2])

final_response = schain.run("Pizza")

print(final_response)



> Entering new LLMChain chain...
Prompt after formatting:
I want to write a new lecture about Pizza, suggest a fancy name for this

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Suggest some topics for this lecture: "The Art and Science of Pizza: A Gastronomic Journey"

> Finished chain.
1. "The History of Pizza: From Its Humble Origins to Global Phenomenon"
2. "Pizza and Culture: How Different Regions Shape Pizza Varieties"
3. "The Art of Pizza Making: From Dough to Toppings"
4. "Pizza and Science: Understanding Yeast Fermentation and Gluten Formation" 
5. "The Health Aspects of Pizza: Nutritional Value and Portion Control"
6. "The Business of Pizza: From Small Pizzerias to Global Chains"
7. "Global Pizza Variations: Understanding Different Styles and Ingredients"
8. "The Role of Pizza in Popular Culture and Media"
9. "Pizza Pairings: An Exploration of Wine, Beer, and Pizza"
10. "Pizza Baking Techniques: From Wood-Fired Ovens to Home Kitchen"
11. "The

In [26]:
from langchain.chains import SequentialChain


# llm=ChatOpenAI(model="gpt-4")
llm=ChatOpenAI(openai_api_base="http://localhost:1234/v1", max_tokens=5000)

# 1
prompt_template_movie_name = PromptTemplate(
    input_variables=["subject"],
    template="I want to watch a movie on {subject},recommend me on some movies"
)

chain1 = LLMChain(llm=llm, prompt=prompt_template_movie_name, output_key="movie_name")

# 2
prompt_template_movie_plot = PromptTemplate(
    input_variables=["movie_name"],
    template="What is the plot for the movie {movie_name}?"
)

chain2 = LLMChain(llm=llm, prompt=prompt_template_movie_plot, output_key="plot")

schain = SequentialChain(
    chains = [chain1, chain2],
    input_variables = ["subject"],
    output_variables = ["movie_name", "plot"],
)

schain("Space")


{'subject': 'Space',
 'movie_name': ' Sure! Here are some great movies set in space that you might enjoy:\n\n1. "Inception" (2010): Directed by Christopher Nolan, this mind-bending sci-fi thriller follows a thief named Dom (played by Leonardo DiCaprio) who is tasked with stealing information from someone\'s subconscious while they dream.\n2. "Interstellar" (2014): Directed by Christopher Nolan, this epic space drama follows a team of astronauts traveling through a wormhole in search of a new home for humanity as Earth is on the brink of destruction.\n3. "Arrival" (2016): Directed by Denis Villeneuve, this sci-fi drama tells the story of a linguist named Louise (played by Amy Adams) who is tasked with communicating with an alien race that has arrived on Earth.\n4. "The Martian" (2015): Directed by Ridley Scott, this epic space drama follows astronaut Mark Watney (played by Matt Damon) as he struggles to survive alone on Mars after his crew is forced to leave him behind.\n5. "E.T. the Ex

**Agents and Tools**

In [27]:
!pip install google-search-results


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [30]:
os.environ['SERPAPI_API_KEY'] = ''

In [31]:
from langchain.agents import AgentType, initialize_agent, load_tools

llm=ChatOpenAI(model="gpt-4")

tools = load_tools(["serpapi", "llm-math"], llm=llm)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# agent.run("Who won the World cup in 2022?")
agent.run("What was the GDP of Israel in 2022 plus one million")



> Entering new AgentExecutor chain...
I need to find out the GDP of Israel in 2022 first, then I can add one million to that figure.
Action: Search
Action Input: "Israel GDP 2022"
Observation: $522,033,000,000
Thought:Now that I have the GDP of Israel for 2022, I can add one million to it.
Action: Calculator
Action Input: 522033000000 + 1000000
Observation: Answer: 522034000000
Thought:I now know the final answer.
Final Answer: The GDP of Israel in 2022 plus one million is $522,034,000,000.

> Finished chain.


'The GDP of Israel in 2022 plus one million is $522,034,000,000.'

In [32]:
!pip install wikipedia

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=d659e0f5eda69343d3a5808386c88e677269f2046b2fe57c77f3a3c158781bc4
  Stored in directory: /Users/iliagerman/Library/Caches/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [34]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm=ChatOpenAI(model="gpt-4")

tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# agent.run("Who won the World cup in 2022?")
response = agent.run("Who is the biggest animal is the world?")
print(response)





> Entering new AgentExecutor chain...
The biggest animal in the world is generally known, but to ensure accuracy, I'll use Wikipedia to confirm.
Action: Wikipedia
Action Input: Biggest animal in the world
Observation: Page: Largest and heaviest animals
Summary: The largest animal currently alive is the blue whale. The maximum recorded weight was 190 tonnes for a specimen measuring 27.6 metres (91 ft), whereas longer ones, up to 33.6 metres (110 ft), have been recorded but not weighed. It is estimated, this individual could have a mass of 250 tonnes. The longest non-colonial animal is the lion's mane jellyfish (36.6m / 120 ft).In 2023, paleontologists estimated that the extinct whale Perucetus, discovered in Peru, may have outweighed the blue whale, with a mass of 85-340 t (84-335 long tons; 94-375 short tonnes). While controversial, estimates for the weight of the sauropod Bruhathkayosaurus suggest it was around 110-170 tons, with the highest estimate being 240 tons, if scaled with P

OutputParserException: Could not parse LLM output: ` To answer this question, we need to use the Wikipedia tool. The action should be "Wikipedia". 
Action Input: "biggest animal in the world"`

In [35]:
llm=ChatOpenAI(openai_api_base="http://localhost:1234/v1", max_tokens=5000)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

response = agent.run("Take a deep breath and think about this step-by-step. Who won the World cup in 2022?")
# response = agent.run("Take a deep breath and think about this step-by-step. Who is the biggest animal is the world?")
print(response)




> Entering new AgentExecutor chain...
 We should start by doing research on animals.
Action: Wikipedia
Action Input: "biggest animal in the world"
Observation: Page: Largest and heaviest animals
Summary: The largest animal currently alive is the blue whale. The maximum recorded weight was 190 tonnes for a specimen measuring 27.6 metres (91 ft), whereas longer ones, up to 33.6 metres (110 ft), have been recorded but not weighed. It is estimated, this individual could have a mass of 250 tonnes. The longest non-colonial animal is the lion's mane jellyfish (36.6m / 120 ft).In 2023, paleontologists estimated that the extinct whale Perucetus, discovered in Peru, may have outweighed the blue whale, with a mass of 85-340 t (84-335 long tons; 94-375 short tonnes). While controversial, estimates for the weight of the sauropod Bruhathkayosaurus suggest it was around 110-170 tons, with the highest estimate being 240 tons, if scaled with Patagotitan, although actual fossil remains no longer exist

KeyboardInterrupt: 

**Memory**

In [36]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

llm=ChatOpenAI(model="gpt-4")

memory = ConversationBufferMemory()


prompt_template_lecture_name = PromptTemplate(
    input_variables=["subject"],
    template="I want to write a new lecture about {subject}, suggest a fancy name for this"
)

chain = LLMChain(llm=llm, prompt=prompt_template_lecture_name, memory=memory ,verbose=True)
response = chain.run("Large Language Model")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
I want to write a new lecture about Large Language Model, suggest a fancy name for this

> Finished chain.
"Decoding the Linguistic Universe: A Deep Dive into Large Language Models"


In [37]:
response = chain.run("Pizza")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
I want to write a new lecture about Pizza, suggest a fancy name for this

> Finished chain.
"Sauced, Sliced, Savored: A Culinary Journey through the World of Pizza"


In [38]:
response = chain.run("Space")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
I want to write a new lecture about Space, suggest a fancy name for this

> Finished chain.
"Cosmic Chronicles: Unraveling the Mysteries of the Universe"


In [69]:
print(chain.memory.buffer)

Human: Large Language Model
AI: "Decoding the Linguistic Universe: A Deep Dive into Large Language Models"
Human: Pizza
AI: "Sauced, Sliced, Savored: A Culinary Journey through the World of Pizza"
Human: Space
AI: "Cosmic Chronicles: Unraveling the Mysteries of the Universe"


In [63]:
from langchain.chains import ConversationChain

In [88]:

from langchain.memory import ConversationBufferWindowMemory
chain2 = ConversationChain(llm=ChatOpenAI(temperature=0.7) )
chain2.memory = ConversationBufferWindowMemory(k=1)

In [91]:
print(chain2.run("Who won the first world cup?"))
print ("------------------------")
print (chain2.memory.buffer)

The winner of the first FIFA World Cup was Uruguay.
------------------------
Human: Who won the first world cup?
AI: The winner of the first FIFA World Cup was Uruguay.


In [92]:
print(chain2.run("How much is 100 + 200?"))
print ("------------------------")
print (chain2.memory.buffer)

100 + 200 equals 300.
------------------------
Human: How much is 100 + 200?
AI: 100 + 200 equals 300.


In [93]:
print(chain2.run("What was the score of that game?"))
print ("------------------------")
print (chain2.memory.buffer)

I'm sorry, I don't have enough context to provide the score of a specific game. Could you please provide more information?
------------------------
Human: What was the score of that game?
AI: I'm sorry, I don't have enough context to provide the score of a specific game. Could you please provide more information?


In [100]:


memory = ConversationBufferWindowMemory(k=3)

chain2 = ConversationChain(llm=ChatOpenAI(temperature=0.7), memory=memory)

In [101]:
print(chain2.run("Who won the first world cup?"))
print ("------------------------")
print (chain2.memory.buffer)

The first FIFA World Cup was held in 1930 in Uruguay. The winner of that tournament was Uruguay. They defeated Argentina 4-2 in the final to become the first ever World Cup champions.
------------------------
Human: Who won the first world cup?
AI: The first FIFA World Cup was held in 1930 in Uruguay. The winner of that tournament was Uruguay. They defeated Argentina 4-2 in the final to become the first ever World Cup champions.


In [102]:
print(chain2.run("How much is 100 + 200?"))
print ("------------------------")
print (chain2.memory.buffer)

100 + 200 equals 300.
------------------------
Human: Who won the first world cup?
AI: The first FIFA World Cup was held in 1930 in Uruguay. The winner of that tournament was Uruguay. They defeated Argentina 4-2 in the final to become the first ever World Cup champions.
Human: How much is 100 + 200?
AI: 100 + 200 equals 300.


In [103]:
print(chain2.run("What was the score of that game?"))
print ("------------------------")
print (chain2.memory.buffer)

In the final of the first FIFA World Cup, Uruguay defeated Argentina with a score of 4-2.
------------------------
Human: Who won the first world cup?
AI: The first FIFA World Cup was held in 1930 in Uruguay. The winner of that tournament was Uruguay. They defeated Argentina 4-2 in the final to become the first ever World Cup champions.
Human: How much is 100 + 200?
AI: 100 + 200 equals 300.
Human: What was the score of that game?
AI: In the final of the first FIFA World Cup, Uruguay defeated Argentina with a score of 4-2.


**Doument Loaders**

In [55]:
!pip install pypdf


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./sample_data/demo.pdf")
pages = loader.load();

In [ ]:
pages[2]